In [1]:
import numpy as np
import keras
import pickle
from keras.utils.io_utils import HDF5Matrix
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import binary_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from keras.models import load_model
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.utils.class_weight import compute_class_weight
import itertools
from functions import encode, decode, plot_confusion_matrix, plot_metrics, plot_images, class_weights_from_labels, f1, plot_f1
%matplotlib inline

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
keras.__version__

'2.1.6'

In [3]:
train_path = '/Users/diegoeduardo/Desktop/Image Gravity Project/Coral Reef Data desk/annotated images balanced/train'
valid_path = '/Users/diegoeduardo/Desktop/Image Gravity Project/Coral Reef Data desk/annotated images balanced/valid'
test_path = '/Users/diegoeduardo/Desktop/Image Gravity Project/Coral Reef Data desk/annotated images balanced/test'

In [4]:
classes = ['2', '13', '16', '27', '28', '29', '33', '39', '45', '54', '59', '64', '66', '67', '71', '126', '127', '137', '239', '240', '241', '245', '248', '253', '273', '400', '655']

In [5]:
train_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(train_path, target_size=(224, 224), 
                                                         classes=classes, batch_size=32)

valid_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(valid_path, target_size=(224, 224), 
                                                         classes=classes, batch_size=32)

test_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(test_path, target_size=(224, 224), 
                                                        classes=classes, batch_size=100)

Found 13500 images belonging to 27 classes.
Found 2700 images belonging to 27 classes.
Found 2700 images belonging to 27 classes.


In [6]:
vgg16_model = load_model('/Users/diegoeduardo/Desktop/Image Gravity Project/Coral Reef Data desk/models/mlb_classifier_reduced.h5', custom_objects={'f1': f1})

In [7]:
vgg16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [10]:
for layer in vgg16_model.layers:
    layer.trainable = True

In [8]:
batch_size = 32
learning_rate = 1e-4
steps_per_epoch = 13500 // batch_size #to go through all examples
val_steps = 2700 // batch_size #32*10 = 320 examples for validation --> do not use all examples --> reduce time of computation
epochs = 20

In [9]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, verbose=1, 
                                               mode='min')
filepath = '/Users/diegoeduardo/Desktop/Coral Reef Data/models/sl_classifier.h5'
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, 
                                             save_weights_only=False, mode='min', period=1)

In [10]:
history = vgg16_model.fit_generator(train_batches, steps_per_epoch=steps_per_epoch, 
                                    epochs=epochs, validation_data=valid_batches, 
                                    validation_steps=val_steps,
                                    callbacks=[checkpoint, early_stopping], verbose=1)

Epoch 1/20
 12/421 [..............................] - ETA: 1:52:46 - loss: 0.2655 - acc: 0.9266 - f1: 0.0277

KeyboardInterrupt: 